## 路径还原

注：存在 86 个重复三元组，提交不知道会不会遇到这些情形
- 比如假设 triple1 即为 en 也为 zh，那么
- en -> zh -> en 和 zh -> zh -> en 都将作为答案

In [1]:
include("../src/CCKS-mKGQA.jl")
include("../src/extractdata.jl")
include("../src/translatedata.jl")
include("../src/ettalign.jl")
# include("../src/deduction.jl")
include("../src/submitpath.jl")
# mkpath("submit")

valid_path

### 初始化

In [2]:
filepath = "ILLs_2"
newILLs = readtuples("EA_data/$filepath/ETT_Pairs.txt"; size=2)
filter!(i->i[1] != "Gregoire_Kayibanda", newILLs);

### 翻译和构建

In [3]:
# ILLs 单词翻译
mt_newILLs = [getindex.(Ref(dict_words), pair) for pair in newILLs]
filter!(i->i[1]!=i[2], unique!(mt_newILLs))
dict_ILLs = ill2dict(mt_newILLs)
ill_valid_ners = [get(dict_ILLs, ner, ner) for ner in last.(mt_valid_ques_ner)]

# ILLs 元组翻译
triple_byills(triple) = triple_byills(triple...)
triple_byills(sub, rel, obj) = (get(dict_ILLs, sub, sub), rel, get(dict_ILLs, obj, obj))
triple_raw2ill(triple...) = triple_byills(triple_byMT(triple...))
ill_triples = triple_raw2ill.(triples)

# 构建边集
edges = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for (sub, rel, obj) in ill_triples
    push!(edges[sub], (rel, obj))
end

# 元组逆向
f(raw_triple) = String.(triple_raw2ill(raw_triple[2:end]))
raw_triples = vcat(tuplejoin.(Ref(("zh",)), zh_triples), tuplejoin.(Ref(("en",)), en_triples))
elt1, elt2 = eltype(raw_triples), typeof(f(raw_triples[1]))
dict_triples_rev = DefaultDict{elt2, Vector{elt1}}(Vector{elt1})
for triple in raw_triples
    push!(dict_triples_rev[f(triple)], triple)
end
println("重复元组的信息")
sort_count(length.(values(dict_triples_rev)))

重复元组的信息


OrderedDict{Int64, Int64} with 4 entries:
  1 => 255733
  2 => 5823
  3 => 30
  4 => 4

### 提交解答

#### 精确匹配

In [45]:
"""提交精确匹配"""
precise_submit(ner, rels, edges, dict) = submit_format.(
    vcat(valid_path.(find_paths(ner, rels, edges), Ref(dict))...))
# function precise_submit(ner, rels, edges, dict)
#     res = String[]
#     for path in find_paths(ner, rels, edges)
#         for rawpath in valid_path(path, dict)
#             push!(res, submit_format(rawpath))
#         end
#     end
#     res
# end

precise_submit (generic function with 1 method)

In [49]:
submit_id = 17
sols = split.(strip.(split(strip(read(open("predict/predict_data_2.txt", "r"), String)), '\n')),'\t')
noans = "<http://dbpedia.org/resource/Francis_Russell,_Marquess_of_Tavistock>#<http://dbpedia.org/property/parents>#<http://dbpedia.org/resource/John_Russell,_4th_Duke_of_Bedford>#<http://zh.dbpedia.org/resource/第四代贝德福德公爵约翰·罗素>#<http://zh.dbpedia.org/property/successor>#<http://zh.dbpedia.org/resource/丹尼尔·芬奇，第八代温奇尔西伯爵>";

fails = String[]
notgood = []
open("submit/submit_$(submit_id).txt", "w") do io
    println(io, "id\tans_path")
    for (i, sol) in enumerate(sols)
        que, rels = sol[1], sol[3:end]
        ner, rawner = ill_valid_ners[i], valid_ques_ner[i][2]
        res = precise_submit(ner, rels, edges, dict_triples_rev)
        if isempty(res)
            push!(fails, que)
            println(io, i-1, '\t', noans)
        else
            betterres = filter(str->occursin(rawner, str), res)
            isempty(betterres) ? push!(notgood, (i, rawner, ner, res)) : (res = betterres)
            println(io, i-1, '\t', first(res))
        end
    end
end
length(fails), length(notgood)

(667, 1)

#### 模糊匹配

In [ ]:
submit_id = 17
sols = split.(strip.(split(strip(read(open("predict/predict_data_2.txt", "r"), String)), '\n')),'\t')
noans = "<http://dbpedia.org/resource/Francis_Russell,_Marquess_of_Tavistock>#<http://dbpedia.org/property/parents>#<http://dbpedia.org/resource/John_Russell,_4th_Duke_of_Bedford>#<http://zh.dbpedia.org/resource/第四代贝德福德公爵约翰·罗素>#<http://zh.dbpedia.org/property/successor>#<http://zh.dbpedia.org/resource/丹尼尔·芬奇，第八代温奇尔西伯爵>";

fails = String[]
notgood = []
open("submit/submit_$(submit_id).txt", "w") do io
    println(io, "id\tans_path")
    for (i, sol) in enumerate(sols)
        que, rels = sol[1], sol[3:end]
        ner, rawner = ill_valid_ners[i], valid_ques_ner[i][2]
        res = precise_submit(ner, rels, edges, dict_triples_rev)
        if isempty(res)
            push!(fails, que)
            println(io, i-1, '\t', noans)
        else
            betterres = filter(str->occursin(rawner, str), res)
            isempty(betterres) ? push!(notgood, (i, rawner, ner, res)) : (res = betterres)
            println(io, i-1, '\t', first(res))
        end
    end
end
length(fails), length(notgood)